In [8]:
# Import dependencies
# numpy for matrix algbera
import numpy as np
# Pandas for data manipulation
import pandas as pd
# matplotlib for data visualization
import matplotlib.pyplot as plt

# Set seed so we get same random allocation on each run of code
np.random.seed(2017)

# Load the IMDB data
educ_data = pd.read_csv("Grade1Students.csv")

# Show structure of data set
educ_data.head()

,g1freelunch,g1absent,g1readscore,g1mathscore,g1listeningscore,g1wordscore
0,1,9,516,578,601,493
1,0,12,451,507,584,436
2,1,4,483,526,529,486
3,1,15,516,505,556,536
4,1,2,433,463,504,426


In [9]:
# Summary statistics for data set (mean, stdev, min, max, etc)
educ_data.describe()

,g1freelunch,g1absent,g1readscore,g1mathscore,g1listeningscore,g1wordscore
count,5550.000000,5550.000000,5550.000000,5550.000000,5550.000000,5550.000000
mean,0.500901,7.421261,521.307207,531.456216,567.824324,514.643063
std,0.500044,7.004582,55.278448,43.151113,33.562973,52.858396
min,0.000000,0.000000,404.000000,404.000000,477.000000,317.000000
25%,0.000000,2.000000,478.000000,502.000000,543.000000,475.000000
50%,1.000000,6.000000,516.000000,529.000000,565.000000,514.000000
75%,1.000000,10.000000,558.000000,562.000000,588.000000,551.000000
max,1.000000,84.000000,651.000000,676.000000,708.000000,601.000000


In [11]:
# We will normalize each feature to have mean 0 and standard deviation 1
# This standardization is done to represent input data on the same scale

# Standardize free lunch status
educ_data["g1freelunch"] = (educ_data["g1freelunch"]-np.mean(educ_data["g1freelunch"]))/np.std(educ_data["g1freelunch"])

# Standardize absences 
educ_data["g1absent"] = (educ_data["g1absent"]-np.mean(educ_data["g1absent"]))/np.std(educ_data["g1absent"])

# Standardize reading score
educ_data["g1readscore"] = (educ_data["g1readscore"]-np.mean(educ_data["g1readscore"]))/np.std(educ_data["g1readscore"])

# Standardize math score
educ_data["g1mathscore"] = (educ_data["g1mathscore"]-np.mean(educ_data["g1mathscore"]))/np.std(educ_data["g1mathscore"])

# Standardize listening score
educ_data["g1listeningscore"] = (educ_data["g1listeningscore"]-np.mean(educ_data["g1listeningscore"]))/np.std(educ_data["g1listeningscore"])

# Standardized word study score
educ_data["g1wordscore"] = (educ_data["g1wordscore"]-np.mean(educ_data["g1wordscore"]))/np.std(educ_data["g1wordscore"])

In [15]:
# Initialize total number of itterations (remember n = 5550)
total_itter = 3*len(educ_data.index)

# Initialize number of output nodes
nodes_num = 3

# Dimension of input data
input_dim = len(educ_data.columns)

# Initialize parameters for learning rate 
learn_init = 0.1

# Step 1: Initialize the weight vectors 
# Randomly generated matrix with entries between [-2,2], each column is a weight vector 
Weight_mat = 4*np.random.rand(input_dim,nodes_num)-2

# Show initialized weight matrix
print("Initialized weight matrix,", Weight_mat)

# Start SOM algorithm itterations
for itter in range(total_itter):
    
    # Initialize distance from weight to chosen point (will be updated in inner loop)
    dist_bmu = float("inf")
    
    # Step 2: Choose data point at random from input data
    
    # Select row index at random
    row_index = np.random.randint(len(educ_data.index))
    
    # Get corresponding data vector
    data_chosen = educ_data.loc[[row_index]]
    
    # Step 3: Find the weight vector that is closest to chosen point
    for node in range(nodes_num):
        
        # Compute euclidean distance from weight vector to chosen point
        dist_neuron = np.linalg.norm(data_chosen-Weight_mat[:,node])
        
        # Save the node with shortest distance of its neuron to chose point
        if dist_neuron < dist_bmu:
            
            # Update distance from weight to chosen point
            dist_bmu = dist_neuron
            
            # Best matching unit (BMU)
            weight_bmu = Weight_mat[:,node]
            index_bmu = node
            
    # Step 4: Define radius of winning neuron neighbourhood 
    # We skip this step because we only have 3 neurons in our application
    
    # Define learning rate
    learn_rate = learn_init*np.exp(-itter/total_itter)
    
    # Step 5: Update weight vectors (w_{t+1} = w_{t} + L(t)*(x_{i} - w_{t}))
    Weight_mat[:,index_bmu] = np.add(weight_bmu,learn_rate*(np.subtract(data_chosen,weight_bmu)))

# Show trained weights
print ("Trained weights from SOM,", Weight_mat)

Initialized weight matrix, [[ 0.29258533 -1.68141927  0.4667529 ]
 [-1.71926387 -0.18965199 -0.15046866]
 [-0.63240642 -0.7812671  -0.67740072]
 [-0.19819022 -0.29255102 -0.25761752]
 [-0.23453879 -1.1255962   1.59932537]
 [-1.23392882 -0.92271883  0.35753994]]
Trained weights from SOM, [[ 0.90212207 -0.19966709 -0.56332296]
 [ 0.09685719 -0.1126318  -0.1866428 ]
 [-1.08109289 -0.10474967  1.29763563]
 [-0.90824374  0.00563766  1.26342202]
 [-0.83703618  0.07549108  1.05139537]
 [-1.10931415  0.02766761  1.09598478]]


In [16]:
# Initialize vector the classifies each student into group 1,2,3
group = np.zeros(len(educ_data.index))
    
# Classify input data
for index, data in educ_data.iterrows():
    
    # Initialize distance from cluster centroid
    dist_cluster = float("inf")
    
    # Find closest weight centroid
    for centroid in range(nodes_num):
        
        # Compute euclidean distance from centroid vector to data point
        dist_centroid = np.linalg.norm(data-Weight_mat[:,centroid])

        # Save centroid that is closest to data piont
        if dist_centroid < dist_cluster:

                # Update distance from weight to chosen point
                dist_cluster = dist_centroid

                # Best matching unit (BMU)
                group[index] = centroid+1
            
# Add group classifier column 
educ_data["group"] = group

# See labeled data (last column contains labels)
educ_data.head()

,g1freelunch,g1absent,g1readscore,g1mathscore,g1listeningscore,g1wordscore,group
0,0.998200,0.225407,-0.096017,1.078720,0.988550,-0.409491,2.0
1,-1.001803,0.653737,-1.271988,-0.566809,0.481993,-1.487941,2.0
2,0.998200,-0.488476,-0.693049,-0.126456,-1.156865,-0.541932,1.0
3,0.998200,1.082066,-0.096017,-0.613161,-0.352334,0.404077,2.0
4,0.998200,-0.774029,-1.597642,-1.586573,-1.901800,-1.677142,1.0


In [18]:
# Let us figure out which group is weak, average, strong

# For group 1:
# Notice the test scores are close to 0 standard deviations away from the mean
# This is likely to be the "average" group
educ_data[educ_data.group == 1].describe()

,g1freelunch,g1absent,g1readscore,g1mathscore,g1listeningscore,g1wordscore,group
count,1709.000000,1709.000000,1709.000000,1709.000000,1709.000000,1709.000000,1709.0
mean,0.778188,0.203853,-0.997276,-0.950488,-0.883719,-1.033823,1.0
std,0.625978,1.211918,0.418782,0.606835,0.591898,0.579465,0.0
min,-1.001803,-1.059582,-2.122306,-2.953984,-2.706331,-3.739441,1.0
25%,0.998200,-0.631252,-1.326264,-1.377985,-1.305852,-1.393340,1.0
50%,0.998200,-0.060146,-1.000610,-0.960808,-0.888688,-1.033856,1.0
75%,0.998200,0.796513,-0.711141,-0.566809,-0.501322,-0.636533,1.0
max,0.998200,10.933648,0.663841,1.078720,1.286524,0.687880,1.0


In [20]:
# For group 2:
# Notice that of students recieving free or reduced lunch much more than average, and
# are absent more than average, and have relatively lower test scores. 
#This is likely to be the "weak" group.
educ_data[educ_data.group == 2].describe()

,g1freelunch,g1absent,g1readscore,g1mathscore,g1listeningscore,g1wordscore,group
count,2416.000000,2416.000000,2416.000000,2416.000000,2416.000000,2416.000000,2416.0
mean,-0.217034,-0.071729,-0.050496,0.009322,0.020750,0.051422,2.0
std,0.976767,0.900481,0.512747,0.590565,0.672129,0.607979,0.0
min,-1.001803,-1.059582,-1.507183,-1.864690,-1.901800,-1.866344,2.0
25%,-1.001803,-0.774029,-0.439763,-0.381397,-0.411929,-0.295969,2.0
50%,-1.001803,-0.202923,-0.041742,0.012603,0.005235,-0.012167,2.0
75%,0.998200,0.368183,0.265820,0.406603,0.362804,0.404077,2.0
max,0.998200,7.935341,2.346384,2.214367,2.746598,1.633888,2.0


In [21]:
# Four group 3:
# The student test scores much higher than on average in this group, students
# are from advantageous backgrounds as lower proportion of students on free
# or reduced price lunch. This is the "gifted" group.
educ_data[educ_data.group == 3].describe()

,g1freelunch,g1absent,g1readscore,g1mathscore,g1listeningscore,g1wordscore,group
count,1425.000000,1425.000000,1425.000000,1425.000000,1425.000000,1425.000000,1425.0
mean,-0.565311,-0.122868,1.281643,1.124114,1.024662,1.152679,3.0
std,0.826401,0.828809,0.632093,0.717906,0.849969,0.501060,0.0
min,-1.001803,-1.059582,-0.439763,-0.891279,-1.097270,-0.409491,3.0
25%,-1.001803,-0.774029,0.772392,0.592015,0.481993,0.687880,3.0
50%,-1.001803,-0.345699,1.242781,1.078720,0.988550,1.179804,3.0
75%,-1.001803,0.225407,1.948363,1.611779,1.495106,1.633888,3.0
max,0.998200,6.793128,2.346384,3.350013,4.176874,1.633888,3.0
